In [1]:
import ee
import geemap
from classification import *
from get_files import *
from riverMask import *

In [2]:
ee.Initialize()

In [50]:
# user inputs
PS_id = 'users/luoyee1997/example/Tallahatchie_Planet'
LS_id = 'users/luoyee1997/example/Tallahatchie_Landsat'
roi_id = 'projects/ee-alpha-luoyee1997/assets/LT'

In [51]:
bands_in_PS = ['b1', 'b2', 'b3', 'b4']
bandnames_PS = ['Blue', 'Green', 'Red', 'NIR']
bands_in_LS = ['SR_B4', 'SR_B3', 'SR_B2', 'SR_B5', 'SR_B6']
bandnames_LS = ['Red', 'Green', 'Blue', 'NIR', 'SWIR1']
roi = load_file(roi_id)
imgLS = load_file(LS_id, bands_in_LS, bandnames_LS)#.clip(roi)
imgPS = load_file(PS_id, bands_in_PS, bandnames_PS)#.clip(roi)

In [40]:
task = ee.batch.Export.table.toAsset(**{
    'collection': ee.FeatureCollection(roi),
    'description': 'Tallahatchie River',
    'assetId': 'users/luoyee1997/example/Tallahatchie_roi'
})
task.start()

In [41]:
classLS = classification(imgLS, thr1=-1, thr2=0, thr3=0.2, thr4=0.3, thr5=0.6, thr6=1, numPts=1000)

In [42]:
waterLS = classLS.eq(1)
#(image, radius=1.5, kernelType='circle', units='pixels', iterations=1, kernel=None)
waterLS1 = close(waterLS, radius=1.5, kernelType='square')
riverLS = noise_removal(waterLS1, 100, 4)

In [48]:
Map = geemap.Map()
vis = {
    'bands': ['Red', 'Green', 'Blue'],
    'min': 0,
    'max': 0.7042,
    'gamma': 2
}
Map.centerObject(imgLS, 12)
Map.addLayer(imgLS.clip(roi), vis, 'LS')

In [47]:
Map.addLayer(classLS.clip(roi), {'min':1, 'max':3, 'palette': ['#0080FF', 'red', 'green']}, 'classLS')

In [45]:
Map.addLayer(waterLS1.clip(roi), {}, 'river')

In [49]:
Map

Map(center=[33.19379318436067, -90.67387865144208], controls=(WidgetControl(options=['position', 'transparent_…